# Setup

## Importing

In [ ]:
import numpy as np
import tensorflow as tf

def _get_time():
    import datetime

    now = datetime.datetime.now() + datetime.timedelta(hours=7)
    return now.strftime('%Y-%m-%d-%H%M')

## Mounting

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd '../gdrive/MyDrive/Capstone/ML'

/gdrive/MyDrive/Capstone/ML


## ELT

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=[0.9,1.1],
    shear_range=180,
    brightness_range=[0.8,1.2],
    # channel_shift_range=150.0,
    horizontal_flip=True,
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
)

In [ ]:
train_dir = './House_Room_Dataset/Training'
validation_dir = './House_Room_Dataset/Validation'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224,224),
)

Found 7994 images belonging to 5 classes.
Found 1068 images belonging to 5 classes.


# Training

## v1 Original

### Hide

#### 1st Run Test Run v1.0.01

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 5

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
)

Epoch 1/5
250/250 [==============================] - 2114s 8s/step - loss: 1.6298 - accuracy: 0.3085 - val_loss: 1.5097 - val_accuracy: 0.3301
Epoch 2/5
250/250 [==============================] - 156s 625ms/step - loss: 1.4253 - accuracy: 0.3734 - val_loss: 1.3727 - val_accuracy: 0.4344
Epoch 3/5
250/250 [==============================] - 156s 625ms/step - loss: 1.3785 - accuracy: 0.4089 - val_loss: 1.2597 - val_accuracy: 0.4976
Epoch 4/5
250/250 [==============================] - 156s 623ms/step - loss: 1.3449 - accuracy: 0.4253 - val_loss: 1.2203 - val_accuracy: 0.5206
Epoch 5/5
250/250 [==============================] - 158s 631ms/step - loss: 1.3070 - accuracy: 0.4567 - val_loss: 1.1569 - val_accuracy: 0.5474


In [ ]:
model.save("./Model/")

INFO:tensorflow:Assets written to: ./Model/assets


#### 2nd Run With Validation v1.0.02

In [ ]:
new_model = tf.keras.models.load_model('./Models/Model-202205131111')

OSError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
new_model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
)

NameError: ignored

In [ ]:
new_model.save('./Models/Model-2205131137')

#### 3rd Run With Augmentation v1.0.03

In [ ]:
model = tf.keras.models.load_model('./Models/Model-2205131137')

In [ ]:
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
)

In [ ]:
model.save(f"./Models/Model-{_get_time()}")

#### 4th Run With Preprocessed Dataset, Recompiled Model v1.0.04

In [ ]:
model = tf.keras.models.load_model('./Models/Model-2022-05-13-1747')

reset_optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=reset_optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
)

In [ ]:
model.save(f"./Models/Model-{_get_time()}")

### 5th Run v1.0.05

In [ ]:
model = tf.keras.models.load_model('./Models/Model-2022-05-15-1631')

In [ ]:
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
)

In [ ]:
model.save(f"./Models/Model-{_get_time()}")

In [ ]:
model = tf.keras.models.load_model('./Models/Model-2022-05-16-1736')

In [ ]:
EPOCHS = 30

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
)

In [ ]:
model.save(f"./Models/Model-{_get_time()}")

## v2 VGG16 Extract Feature

In [ ]:
ckpt_dir = './Models/Model-v2-vgg16'

callback_ckpt = tf.keras.callbacks.ModelCheckpoint(
    ckpt_dir,
    monitor='val_loss',
    save_best_only=True,
)

### Hide

#### 1st Run

In [ ]:
vgg16_extractor = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

for layer in vgg16_extractor.layers[0:-4]:
    layer.trainable = False

last_desired_layer = vgg16_extractor.get_layer('fc2')
last_output = last_desired_layer.output

classifier = tf.keras.layers.Dense(5, activation='softmax')(last_output)

model = tf.keras.Model(inputs=vgg16_extractor.input, outputs=classifier)

model.summary()

553476096/553467096 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#@title Default title text
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

In [ ]:
#@title Default title text
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

In [ ]:
#@title Default title text
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

In [ ]:
#@title Default title text
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

In [ ]:
#@title Default title text
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

#### 2nd Run

In [ ]:
model = tf.keras.models.load_model(ckpt_dir)

# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

In [ ]:
EPOCHS = 40

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

### 3rd Run

In [ ]:
model = tf.keras.models.load_model(ckpt_dir)

# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

In [ ]:
EPOCHS = 40

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

Epoch 1/40
250/250 [==============================] - 2560s 10s/step - loss: 0.7168 - accuracy: 0.7210 - val_loss: 0.6944 - val_accuracy: 0.7959
Epoch 2/40
250/250 [==============================] - 194s 777ms/step - loss: 0.7233 - accuracy: 0.7225 - val_loss: 0.6399 - val_accuracy: 0.8146
Epoch 3/40
250/250 [==============================] - 186s 743ms/step - loss: 0.7165 - accuracy: 0.7228 - val_loss: 0.6889 - val_accuracy: 0.7996
Epoch 4/40
250/250 [==============================] - 171s 685ms/step - loss: 0.7108 - accuracy: 0.7252 - val_loss: 0.7036 - val_accuracy: 0.7903
Epoch 5/40
250/250 [==============================] - 180s 719ms/step - loss: 0.7127 - accuracy: 0.7218 - val_loss: 0.6282 - val_accuracy: 0.8193
Epoch 6/40
250/250 [==============================] - 181s 724ms/step - loss: 0.7213 - accuracy: 0.7200 - val_loss: 0.7146 - val_accuracy: 0.7743
Epoch 7/40
250/250 [==============================] - 172s 689ms/step - loss: 0.7014 - accuracy: 0.7243 - val_loss: 0.6607 - 

In [ ]:
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

Epoch 1/10
250/250 [==============================] - 2631s 10s/step - loss: 0.7096 - accuracy: 0.7285 - val_loss: 0.6897 - val_accuracy: 0.8155
Epoch 2/10
250/250 [==============================] - 192s 768ms/step - loss: 0.7001 - accuracy: 0.7314 - val_loss: 0.6234 - val_accuracy: 0.8146
Epoch 3/10
250/250 [==============================] - 186s 745ms/step - loss: 0.7209 - accuracy: 0.7247 - val_loss: 0.6269 - val_accuracy: 0.8052
Epoch 4/10
250/250 [==============================] - 173s 691ms/step - loss: 0.6986 - accuracy: 0.7324 - val_loss: 0.6501 - val_accuracy: 0.8052
Epoch 5/10
250/250 [==============================] - 180s 721ms/step - loss: 0.6974 - accuracy: 0.7318 - val_loss: 0.6230 - val_accuracy: 0.8099
Epoch 6/10
250/250 [==============================] - 184s 738ms/step - loss: 0.6949 - accuracy: 0.7336 - val_loss: 0.6557 - val_accuracy: 0.7968
Epoch 7/10
250/250 [==============================] - 179s 718ms/step - loss: 0.6951 - accuracy: 0.7368 - val_loss: 0.6156 - 

In [ ]:
EPOCHS = 10

model.fit(
    train_generator,
    epochs = EPOCHS,
    validation_data=validation_generator,
    callbacks=[callback_ckpt]
)

Epoch 1/10
250/250 [==============================] - 173s 689ms/step - loss: 0.6852 - accuracy: 0.7352 - val_loss: 0.6829 - val_accuracy: 0.8174
Epoch 2/10
250/250 [==============================] - 170s 680ms/step - loss: 0.6914 - accuracy: 0.7349 - val_loss: 0.6602 - val_accuracy: 0.7865
Epoch 3/10
250/250 [==============================] - 170s 679ms/step - loss: 0.6829 - accuracy: 0.7328 - val_loss: 0.6380 - val_accuracy: 0.8043
Epoch 4/10
250/250 [==============================] - 171s 682ms/step - loss: 0.6910 - accuracy: 0.7392 - val_loss: 0.6168 - val_accuracy: 0.8109
Epoch 5/10
250/250 [==============================] - 180s 718ms/step - loss: 0.6891 - accuracy: 0.7344 - val_loss: 0.6098 - val_accuracy: 0.8127
Epoch 6/10
250/250 [==============================] - 191s 764ms/step - loss: 0.6944 - accuracy: 0.7290 - val_loss: 0.5829 - val_accuracy: 0.8240
Epoch 7/10
250/250 [==============================] - 178s 712ms/step - loss: 0.6686 - accuracy: 0.7452 - val_loss: 0.6762 -

# Testing

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()
dir = !pwd
dir = dir[0]

for fn in uploaded.keys():
    # predicting images
    path= f"{dir}/{fn}"
    img=image.load_img(path, target_size=(224, 224))
    
    x=image.img_to_array(img)
    x /= 255
    x=np.expand_dims(x, axis=0)
    images = np.vstack([x])
    
    classes = model.predict(images, batch_size=10)
    
    imax = np.argmax(classes[0])
    print(classes[0])

    if imax == 0:
        print("Bathroom")
    elif imax == 1:
        print("Bedroom")
    elif imax == 2:
        print("Dinning")
    elif imax == 3:
        print("Kitchen")
    elif imax == 4:
        print("Livingroom")

In [ ]:
testing_dir = './House_Room_Dataset/Testing'

testing_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
)

testing_generator = testing_datagen.flow_from_directory(
    testing_dir,
    target_size=(224,224),
)

Found 499 images belonging to 5 classes.


In [ ]:
y_pred = model.predict(testing_generator)
y_pred = np.apply_along_axis(lambda x: np.argmax(x), 1, y_pred)

y_true = testing_generator.labels

tf.math.confusion_matrix(
    y_true,
    y_pred,
    num_classes=5,
)

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[23, 22, 13, 19, 23],
       [16, 23, 16, 19, 26],
       [22, 21, 18, 24, 15],
       [21, 18, 22, 23, 15],
       [25, 14, 15, 18, 28]], dtype=int32)>

In [ ]:
model.evaluate(
    testing_generator
)

16/16 [==============================] - 266s 16s/step - loss: 0.5007 - accuracy: 0.8216


[0.5007370710372925, 0.8216432929039001]

# Export To TFL

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('./Models/Model-v2-vgg16') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('prod_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
import matplotlib
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# https: // stackoverflow.com/questions/53684971/assertion-failed-flask-server-stops-after-script-is-run
matplotlib.use('Agg')

def get_category(img):
    """Write a Function to Predict the Class Name
    Args:
        img [jpg]: image file with 3 color channels
    Returns:
        [str]: Prediction
    """
    # Read an image from a file into a numpy array
    img = mpimg.imread(img)
    # Convert to float32
    img = tf.cast(img, tf.float32)
    # Resize to 224x224 (size the model is expecting)
    img = tf.image.resize(img, [224, 224])
    # Expand img dimensions from (224, 224, 3) to (1, 224, 224, 3) for set_tensor method call
    img = np.expand_dims(img, axis=0)

    tflite_model_file = './prod_model.tflite'

    with open(tflite_model_file, 'rb') as fid:
        tflite_model = fid.read()

    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    
    prediction = []
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    prediction.append(interpreter.get_tensor(output_index))
    
    predicted_label = [np.argmax(_prediction) for _prediction in prediction]
    class_names = ['Bathroom', 'Bedroom', 'Dinning', 'Kitchen', 'Livingroom']
    predicted_label = [class_names[index] for index in predicted_label]

    return predicted_label


# def plot_category(img):
def plot_category(img, current_time):
    """Plot the input image. Timestamp used to help Flask grab the correct image.
    Args:
        img [jpg]: image file
        current_time: timestamp
    """
    # Read an image from a file into a numpy array
    img = mpimg.imread(img1)
    # Remove the plotting ticks
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)
    # To make sure Flask grabs the correct image to plot
    strFile = f'static/images/output_{current_time}.png'
    if os.path.isfile(strFile):
        os.remove(strFile)
    # Save the image with the file name that result.html is using as its img src
    plt.savefig(strFile)

In [ ]:
dir = '/gdrive/MyDrive/Capstone/ML/House_Room_Dataset/Testing/Bathroom/BATH_23.jpg'
get_category(dir)

['Bathroom']